In [1]:
import requests

from bs4 import BeautifulSoup

import pandas as pd

import numpy as np

In [16]:
url = 'https://tabiturient.ru/vuzcost/'

data = requests.get(url).text

soup = BeautifulSoup(data, 'html.parser')


## Получение ссылок на страницу вуза

In [20]:
uni_links = soup.find('div', {'class': 'obramtop100'}).find_all('a')

links = [i['href'] for i in uni_links]

## Получение средней стоимости обучения


In [31]:
gen_info = []
for i in uni_links:
    gen_info.append([j.text for j in i.find('table', {'class': 'listtop100'}).find_all('td')])

In [32]:
gen_info = [i[-2] for i in gen_info]

In [34]:
gen_prices = [int((i.replace('руб./год', '')).replace(' ','')) for i in gen_info]

## Получение ссылок на инфу об общежитиях вуза

In [37]:
links = [i.replace('proxodnoi', 'about') for i in links]

In [76]:
links_dorm = [i.replace('about', 'obsh') for i in links]

In [39]:
first_url = 'https://tabiturient.ru/vuzu/mgimo/about/'

data1 = requests.get(first_url).text

soup1 = BeautifulSoup(data1, 'html.parser')

## Функция для получения наименований вуза


In [151]:
def find_names(soup):
    try:
        names = [j.text for j in soup.find('div' ,{'class': 'p40 pm40 obram obramvuz1'}).find_all('h2', {'class': 'font4m fontshadow'})]
        return names
    except Exception:
        return [np.nan]*2
    

## Функция для получения локации вуза

In [136]:
def get_location(soup):
    try:
        location = [j.text for j in soup.find('div' ,{'class': 'p40 pm40 obram obramvuz1'}).find_all('b', {'class': 'font1'})]
        return [location[0]]
    except Exception:
        return [np.nan]
    

## Функция для получения информации о структуре вуза

In [135]:
def get_structur(soup):
    try:
        struc = [j.text for j in soup.find('div', {'class': 'p20 obram'}).find_all('b', {'class': 'font5'})]
    
        return struc[:-2]
    except Exception:
        return [np.nan] * 4
        

## Функция для получения оценок студентов различных показателей

In [163]:
def get_marks(soup):
    try:
        marks = [i.text for i in soup.find_all('div', {'class': 'cirdesc1-5'})]
        
        if len(marks) == 0:
            return [np.nan] * 6
        elif len(marks) == 6:
            return marks
    except Exception:
        return [np.nan] * 6

## Функция для получения расходов

In [170]:
def get_expenses(soup):
    try:
        expenses =  [i.text for i in soup.find_all('b', {'class': 'font4'})]
        if len(expenses) == 0:
            return [np.nan] * 2
        elif len(expenses) == 1:
            return expenses + [np.nan]
        elif len(expenses[:2]) == 2:
            return expenses[:2]
    except Exception:
        return [np.nan] * 2

## Функция для получения рейтинга

In [132]:
def get_rating(soup):
    try:
        rating = [i.text for i in soup.find('td', {'class': 'ocenka'}).find_all('b', {'class': 'font5'})]
    
        return rating
    except Exception:
        return [np.nan]

## Функция для приготовления супа

In [75]:
def get_soup(link):
    data = requests.get(link).text
    
    soup = BeautifulSoup(data, 'html.parser')
    
    return soup

In [115]:
df = pd.DataFrame({'Name': [], 'brief_name': [], 'location': [], 'num_subject': [], 'num_educat_prog': [], 'num_budget_pla': [], 'aver_score_for_bugd': [], 'universities_conditions': [], 'locations_availibility': [], 'educ_quality': [], 'administration_qual': [], 'rat_add_activities': [], 'food_quality': [], 'lunch_price': [], 'road_costs_per_month': [], 'av_price_for_educ': [],'Dorm': [], 'Dorm_rating': []})

In [125]:
def get_dorm(url):
    data = requests.get(url).text
    
    soup = BeautifulSoup(data, 'html.parser')
    
    info = [i.text for i in soup.find('div', {'class': 'p40 pm40'}).find_all('b')]
    
    final = []
    
    if 'есть' in info[0]:
        final.append('Yes')
    else:
        final.append('No')
        
    info[-1] = (info[-1].replace('/10', ''))
    
    try:
        final.append(float(info[-1]))
    except Exception:
        final.append(np.nan)
        

    
    return final

In [149]:
def companate(url3, url4):
    soup = get_soup(url3)
    
    
    names = find_names(soup)
    
    location = get_location(soup)
    
    structure = get_structur(soup)
    
    marks = get_marks(soup)
    
    expenses = get_expenses(soup)
    
    rating = get_rating(soup)
    
    dorm = get_dorm(url4)
    
    final = names + location + structure + marks + expenses + rating + dorm
    
    return final

In [ ]:
for i,j in zip(links, links_dorm):
    print(i)
    df.loc[len(df)] = companate(i,j)